# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse    Shipments  \
0  Jul 19 2022  8:30AM  242155        10      8291639   
1  Jul 18 2022  4:37PM  242159        20  8173919-FLS   
2  Jul 18 2022  4:12PM  242158        16      8292185   
3  Jul 18 2022  4:12PM  242158        16      8292186   
4  Jul 18 2022  4:12PM  242158        16      8292191   
5  Jul 18 2022  4:12PM  242158        16      8292197   
6  Jul 18 2022  4:11PM  242157        16      8292173   
7  Jul 18 2022  3:52PM  242146        16      8290869   
8  Jul 18 2022  3:52PM  242146        16      8291446   
9  Jul 18 2022  3:37PM  242147        21       621119   

                             Customer ShipmentStatus  
0                      ClearSpec, LLC       Released  
1            ACI Healthcare USA, Inc.       Released  
2  Sartorius Lab Products and Service       Released  
3  Sartorius Lab Products and Service       Released  
4  Sartorius Lab Products and Service       Released  
5  Sartorius Lab Products and Service       Released  
6      Sartorius Bioprocess Solutions       Released  
7      Sartorius Bioprocess Solutions       Released  
8      Sartorius Bioprocess Solutions       Released  
9                   NBTY Global, Inc.       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
45  242147       Released          1
46  242155       Released          1
47  242157       Released          1
48  242158       Released          4
49  242159       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
242147                NaN        NaN       1.0
242155                NaN        NaN       1.0
242157                NaN        NaN       1.0
242158                NaN        NaN       4.0
242159                NaN        NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
241900                4.0        2.0       6.0
241972                2.0        8.0       4.0
242005                0.0        0.0       3.0
242007                0.0        0.0       1.0
242008                0.0        0.0       1.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
241900                  4          2         6
241972                  2          8         4
242005                  0          0         3
242007                  0          0         1
242008                  0          0         1

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               241900          4          2         6
1               241972          2          8         4
2               242005          0          0         3
3               242007          0          0         1
4               242008          0          0         1

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               241900         4         2        6
1               241972         2         8        4
2               242005                            3
3               242007                            1
4               242008                            1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                            Customer
0   Jul 19 2022  8:30AM  242155        10                      ClearSpec, LLC
1   Jul 18 2022  4:37PM  242159        20            ACI Healthcare USA, Inc.
2   Jul 18 2022  4:12PM  242158        16  Sartorius Lab Products and Service
6   Jul 18 2022  4:11PM  242157        16      Sartorius Bioprocess Solutions
7   Jul 18 2022  3:52PM  242146        16      Sartorius Bioprocess Solutions
9   Jul 18 2022  3:37PM  242147        21                   NBTY Global, Inc.
10  Jul 18 2022  3:22PM  242139        20            ACI Healthcare USA, Inc.
78  Jul 18 2022  3:10PM  242096        10                    Eywa Pharma Inc.
91  Jul 18 2022  3:05PM  242137        21                   NBTY Global, Inc.
92  Jul 18 2022  3:03PM  242136        19             GUSA Granules USA, Inc.

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                            Customer  \
0  Jul 19 2022  8:30AM  242155        10                      ClearSpec, LLC   
1  Jul 18 2022  4:37PM  242159        20            ACI Healthcare USA, Inc.   
2  Jul 18 2022  4:12PM  242158        16  Sartorius Lab Products and Service   
3  Jul 18 2022  4:11PM  242157        16      Sartorius Bioprocess Solutions   
4  Jul 18 2022  3:52PM  242146        16      Sartorius Bioprocess Solutions   
5  Jul 18 2022  3:37PM  242147        21                   NBTY Global, Inc.   
6  Jul 18 2022  3:22PM  242139        20            ACI Healthcare USA, Inc.   
7  Jul 18 2022  3:10PM  242096        10                    Eywa Pharma Inc.   
8  Jul 18 2022  3:05PM  242137        21                   NBTY Global, Inc.   
9  Jul 18 2022  3:03PM  242136        19             GUSA Granules USA, Inc.   

  Completed Executing Released  
0                            1  
1                            1  
2                            4  
3                            1  
4                            2  
5                            1  
6                           68  
7                   4        9  
8                            1  
9                            1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                            Customer  \
0  Jul 19 2022  8:30AM  242155        10                      ClearSpec, LLC   
1  Jul 18 2022  4:37PM  242159        20            ACI Healthcare USA, Inc.   
2  Jul 18 2022  4:12PM  242158        16  Sartorius Lab Products and Service   
3  Jul 18 2022  4:11PM  242157        16      Sartorius Bioprocess Solutions   
4  Jul 18 2022  3:52PM  242146        16      Sartorius Bioprocess Solutions   
5  Jul 18 2022  3:37PM  242147        21                   NBTY Global, Inc.   
6  Jul 18 2022  3:22PM  242139        20            ACI Healthcare USA, Inc.   
7  Jul 18 2022  3:10PM  242096        10                    Eywa Pharma Inc.   
8  Jul 18 2022  3:05PM  242137        21                   NBTY Global, Inc.   
9  Jul 18 2022  3:03PM  242136        19             GUSA Granules USA, Inc.   

  Released Executing Completed  
0        1                      
1        1                      
2        4                      
3        1                      
4        2                      
5        1                      
6       68                      
7        9         4            
8        1                      
9        1

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                            Customer  \
0  Jul 19 2022  8:30AM  242155        10                      ClearSpec, LLC   
1  Jul 18 2022  4:37PM  242159        20            ACI Healthcare USA, Inc.   
2  Jul 18 2022  4:12PM  242158        16  Sartorius Lab Products and Service   
3  Jul 18 2022  4:11PM  242157        16      Sartorius Bioprocess Solutions   
4  Jul 18 2022  3:52PM  242146        16      Sartorius Bioprocess Solutions   

  Released Executing Completed  
0        1                      
1        1                      
2        4                      
3        1                      
4        2

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse                            Customer  \
0  Jul 19 2022  8:30AM  242155        10                      ClearSpec, LLC   
1  Jul 18 2022  4:37PM  242159        20            ACI Healthcare USA, Inc.   
2  Jul 18 2022  4:12PM  242158        16  Sartorius Lab Products and Service   
3  Jul 18 2022  4:11PM  242157        16      Sartorius Bioprocess Solutions   
4  Jul 18 2022  3:52PM  242146        16      Sartorius Bioprocess Solutions   

   Released  Executing  Completed  
0       1.0        NaN        NaN  
1       1.0        NaN        NaN  
2       4.0        NaN        NaN  
3       1.0        NaN        NaN  
4       2.0        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                            Customer  \
0  Jul 19 2022  8:30AM  242155        10                      ClearSpec, LLC   
1  Jul 18 2022  4:37PM  242159        20            ACI Healthcare USA, Inc.   
2  Jul 18 2022  4:12PM  242158        16  Sartorius Lab Products and Service   
3  Jul 18 2022  4:11PM  242157        16      Sartorius Bioprocess Solutions   
4  Jul 18 2022  3:52PM  242146        16      Sartorius Bioprocess Solutions   

   Released  Executing  Completed  
0       1.0        0.0        0.0  
1       1.0        0.0        0.0  
2       4.0        0.0        0.0  
3       1.0        0.0        0.0  
4       2.0        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         1936440      28.0       13.0       20.0
102         242008       1.0        0.0        0.0
12          242135       0.0        1.0        0.0
16          726461       7.0        0.0        0.0
19          968514      11.0       10.0        5.0
20         1936441     143.0       18.0        8.0
21         3389508      14.0        0.0        0.0
22          484232       2.0        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  1936440      28.0       13.0       20.0
1       102   242008       1.0        0.0        0.0
2        12   242135       0.0        1.0        0.0
3        16   726461       7.0        0.0        0.0
4        19   968514      11.0       10.0        5.0
5        20  1936441     143.0       18.0        8.0
6        21  3389508      14.0        0.0        0.0
7        22   484232       2.0        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10      28.0       13.0       20.0
1       102       1.0        0.0        0.0
2        12       0.0        1.0        0.0
3        16       7.0        0.0        0.0
4        19      11.0       10.0        5.0
5        20     143.0       18.0        8.0
6        21      14.0        0.0        0.0
7        22       2.0        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   28.0
1       102  Released    1.0
2        12  Released    0.0
3        16  Released    7.0
4        19  Released   11.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10  102   12   16    19     20    21   22
Status                                                
Completed  20.0  0.0  0.0  0.0   5.0    8.0   0.0  0.0
Executing  13.0  0.0  1.0  0.0  10.0   18.0   0.0  0.0
Released   28.0  1.0  0.0  7.0  11.0  143.0  14.0  2.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10  102   12   16    19     20    21   22
0          Completed  20.0  0.0  0.0  0.0   5.0    8.0   0.0  0.0
1          Executing  13.0  0.0  1.0  0.0  10.0   18.0   0.0  0.0
2           Released  28.0  1.0  0.0  7.0  11.0  143.0  14.0  2.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10  102   12   16    19     20    21   22
0  Completed  20.0  0.0  0.0  0.0   5.0    8.0   0.0  0.0
1  Executing  13.0  0.0  1.0  0.0  10.0   18.0   0.0  0.0
2   Released  28.0  1.0  0.0  7.0  11.0  143.0  14.0  2.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()